In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import anndata as anndata
import pathlib
import sys
import decoupler as dc
import scipy
import seaborn as sns

In [ ]:
sample_name = 'OMIX002713'
use_fcgr1a = True

In [ ]:
def calculate_and_plot_score(adata: anndata.AnnData, gene_list, score_name, **kwargs):
    sc.tl.score_genes(ad, gene_list = gene_list, score_name=score_name)
    sc.pl.spatial(ad, color = score_name, img_key=None, size = 1.6, save = '_{}.png'.format(score_name), return_fig = False, **kwargs)


In [ ]:
database_path = pathlib.Path('../../../shared/databases/')

In [ ]:
if use_fcgr1a:
    MAC_GENES = ['CD163', 'LYZ', 'FCGR1A'] # APOE'CD68' is expressed almost everywhere: 'MRC1' not enough
else:
    MAC_GENES = ['CD163', 'LYZ']

numof_genes = 150
KOvsWT_down_genelist = pd.read_csv(database_path / '{}_KO--over--WT_Down-Regulated.csv'.format(numof_genes), header=0).gene_name.str.upper()
KOvsWT_up_genelist = pd.read_csv(database_path / '{}_KO--over--WT_Up-Regulated.csv'.format(numof_genes), header=0).gene_name.str.upper()
KOvsWT_down_genelist = KOvsWT_down_genelist.str.upper()
KOvsWT_up_genelist = KOvsWT_up_genelist.str.upper()


In [ ]:
print(len(KOvsWT_down_genelist))

In [ ]:
sample_folder = pathlib.Path('../data/raw')

path_to_results = pathlib.Path('../') / 'results'
path_to_results.mkdir(parents=True, exist_ok=True)
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white', figsize = (4,4), dpi_save=300, frameon = False, vector_friendly = False)
sc.settings.figdir = path_to_results
spatial_dot_size = 1.6
spot_size = 1.6
spatial_transparency = 1

In [ ]:
adata = sc.read(sample_folder / "OMIX002713.h5ad")

In [ ]:
len(adata.obs['orig.ident'].cat.categories)

In [ ]:
adata.obs['orig.ident'].cat.categories

In [ ]:
sc.pl.umap(adata, color = ['celltype', 'labels', 'histology'])

In [ ]:
sc.pl.umap(adata, color = MAC_GENES)

In [ ]:
sc.pl.umap(adata, color = MAC_GENES)

In [ ]:
sc.tl.score_genes(adata, gene_list=KOvsWT_down_genelist, score_name='KOvsWT_down_score')

In [ ]:
sc.tl.score_genes(adata, gene_list=KOvsWT_up_genelist, score_name='KOvsWT_up_score')

In [ ]:
sc.pl.umap(adata, color = 'KOvsWT_down_score', cmap='Oranges')

In [ ]:
sc.pl.umap(adata, color = 'KOvsWT_up_score', cmap='Oranges')

In [ ]:
adata.obs['KOvsWT_up_minus_down_score'] = adata.obs['KOvsWT_up_score'] - adata.obs['KOvsWT_down_score']

In [ ]:
vmax = adata.obs['KOvsWT_up_minus_down_score'].abs().quantile(0.99)
vmin = -vmax
sc.pl.umap(adata, color = 'KOvsWT_up_minus_down_score', vmax = vmax, vmin = vmin, cmap='bwr')

# Fig 3F

In [ ]:
df = sc.get.obs_df(adata, keys = ['SPP1', 'CXCL10', 'CD74'])

In [ ]:
#define conditions
conditions = [
    (df['CXCL10'] > 0) & (df['SPP1'] > 0),
    (df['CXCL10'] > 0) & (df['SPP1'] == 0),
    (df['CXCL10'] == 0)& (df['SPP1'] > 0),
    (df['CXCL10'] == 0)& (df['SPP1'] == 0),
]

#define results
results = ['CXCL10 & SPP1', 'CXCL10', 'SPP1', 'none']

#create new column based on conditions in column1 and column2
adata.obs['CXCL10_SPP1'] = np.select(conditions, results)

In [ ]:
Cxcl10_Spp1_colors= {'CXCL10':'#40E0D0', 'SPP1':'#FF1493', 'CXCL10 & SPP1': '#5D29A6', 'none': '#F2F2F2'}

In [ ]:
sc.pl.violin(adata, keys='KOvsWT_up_minus_down_score', groupby='CXCL10_SPP1',  palette = Cxcl10_Spp1_colors, stripplot = False, inner = 'box', rotation = 45)

In [ ]:
#define conditions
conditions = [
    (df['CD74'] > 0) & (df['SPP1'] > 0),
    (df['CD74'] > 0) & (df['SPP1'] == 0),
    (df['CD74'] == 0)& (df['SPP1'] > 0),
    (df['CD74'] == 0)& (df['SPP1'] == 0),
]

#define results
results = ['CD74 & SPP1', 'CD74', 'SPP1', 'none']

#create new column based on conditions in column1 and column2
adata.obs['CD74_SPP1'] = np.select(conditions, results)

In [ ]:
CD74_SPP1_colors= {'CD74':'#40E0D0', 'SPP1':'#FF1493', 'CD74 & SPP1': '#5D29A6', 'none': '#F2F2F2'}

In [ ]:
sc.pl.violin(adata, keys='KOvsWT_up_minus_down_score', groupby='CD74_SPP1', palette = CD74_SPP1_colors, stripplot = False, inner = 'box',rotation = 90)

In [ ]:
msigdb = dc.get_resource('MSigDB')
msigdb

In [ ]:
msigdb['collection'].unique()

In [ ]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection']=='hallmark']

# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]
msigdb

In [ ]:
msigdb.geneset = msigdb.geneset.str.replace('HALLMARK_', '')

In [ ]:
msigdb

In [ ]:
dc.run_aucell(adata, msigdb, source='geneset', target='genesymbol', use_raw=False)

In [ ]:
terms = msigdb.geneset.unique()

In [ ]:
adata.obsm['aucell_estimate']

In [ ]:
adata.obs[terms] = adata.obsm["aucell_estimate"][terms]

In [ ]:
important_terms = ['TNFA_SIGNALING_VIA_NFKB', 'INFLAMMATORY_RESPONSE', 'ALLOGRAFT_REJECTION',
 'INTERFERON_GAMMA_RESPONSE',
 'INTERFERON_ALPHA_RESPONSE',
 'IL6_JAK_STAT3_SIGNALING']

In [ ]:
sc.pl.violin(adata, keys=important_terms, groupby='CXCL10_SPP1',  palette = Cxcl10_Spp1_colors, stripplot = False, inner = 'box', rotation = 45)

In [ ]:
zscore_important_terms = [ term + '_zscore' for term in important_terms]

In [ ]:
for term, zscore_term in zip(important_terms, zscore_important_terms):
    adata.obs[zscore_term] = scipy.stats.zscore(adata.obs[term])

In [ ]:
sc.pl.violin(adata, keys=zscore_important_terms, groupby='CXCL10_SPP1',  palette = Cxcl10_Spp1_colors, stripplot = False, inner = 'box', rotation = 45)

categories_order = sorted(ad_mac_tumor.obs['sample'].cat.categories.tolist(), key = lambda x: x.split('_Day')[1]) # sort by day

sc.pl.matrixplot(adata_tc, zscore_important_terms, categories_order= categories_order, groupby='sample', vcenter = 0, swap_axes=True, cmap='coolwarm')

sc.pl.dotplot(adata_tc, zscore_important_terms, categories_order= categories_order, groupby='sample', vcenter = 0, swap_axes=True, cmap='coolwarm')

In [ ]:
important_terms_zscores_samples = sc.get.obs_df(adata, keys=zscore_important_terms + ['CXCL10_SPP1'])

In [ ]:
important_terms_zscores_samples = important_terms_zscores_samples[important_terms_zscores_samples['CXCL10_SPP1'].isin(['CXCL10', 'SPP1'])]
important_terms_zscores_samples['CXCL10_SPP1'] = important_terms_zscores_samples['CXCL10_SPP1'].cat.remove_unused_categories()

In [ ]:
df_long = important_terms_zscores_samples.melt(id_vars='CXCL10_SPP1', var_name='Pathway', value_name='Zscore')

In [ ]:
ci = 95
alpha = 1 - (ci / 100)

summary = df_long.groupby(['Pathway', 'CXCL10_SPP1']).agg(
    mean=('Zscore', 'mean'),
    std=('Zscore', 'std'),
    count=('Zscore', 'count')
).reset_index()

summary['sem'] = summary['std'] / np.sqrt(summary['count'])

# Compute t critical value and CI dynamically
summary['t_crit'] = summary['count'].apply(lambda n: scipy.stats.t.ppf(1 - alpha / 2, df=n - 1))
summary['ci'] = summary['sem'] * summary['t_crit']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for pathway in summary['Pathway'].unique():
    data = summary[summary['Pathway'] == pathway]
    
    plt.figure(figsize=(6, 4))
    sns.barplot(data=data, y='CXCL10_SPP1', x='mean', errorbar=None, palette=Cxcl10_Spp1_colors)
    
    # Add error bars
    plt.errorbar(
        y=np.arange(len(data)),
        x=data['mean'],
        xerr=data['ci'],
        fmt='none',
        color='black',
        capsize=5
    )
    
    plt.title(f'{pathway} Z-Score by Group ({ci}% CI)')
    plt.ylabel('Mean Z-Score')
    plt.tight_layout()
    plt.savefig(path_to_results / 'glioblastoma_barplot_{}.pdf'.format(pathway))
    plt.show()
    